In [2]:
import warnings


warnings.filterwarnings("ignore")

import nltk
from nltk import FreqDist
nltk.download('stopwords')

# !pip install pyLDAvis

import pandas as pd
# pd.set_option("display.max_colwidth", 200)
import numpy as np
import json
import re
import gzip
import spacy

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from gensim.models.coherencemodel import CoherenceModel

from nltk.corpus import stopwords

import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

from nltk.stem import WordNetLemmatizer

from tqdm import tqdm

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\Users\tanma\Anaconda3\lib\site-packages\matplotlib\colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [3]:
import json

In [4]:
from gensim.test.utils import datapath

In [5]:
import random

## Load all the required data

In the real world this will be equivalent to querying a table

In [6]:
inter_df = pd.read_csv('./data/interactions.csv')

In [7]:
engaged_df = pd.read_csv("./data/engaged_df.csv")

In [8]:
engaged_df.head()

,book_id,user_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,clean_reviews
0,5805,4cbecbc15af3db041a8e0f594c642bb5,58f2301bd2d4bbfc1b51e4e5fb161cfe,5,"Remember, remember, the fifth of November. Thi...",Wed Jun 13 17:55:53 -0700 2012,Mon Jan 30 05:58:01 -0800 2017,Thu Jun 14 00:00:00 -0700 2012,NaN,7,0,"['remember', 'remember', 'fifth', 'part', 'quo..."
1,5805,49cc59f1c479d698507627b401d47ecf,761a17f52538341a085b629a316204a1,4,Tinha apontado este livro como um dos que tinh...,Sun Jan 05 03:27:10 -0800 2014,Tue Oct 28 15:58:20 -0700 2014,Tue Oct 28 15:58:20 -0700 2014,NaN,1,0,"['tinha', 'visto', 'portman', 'hora', 'livro',..."
2,5805,5f03864c758bfceb6d7d5e93eeb20044,ac54c3ce0c9f660c03881b0668f79c60,5,Review coming soon! www.youtube.com/ReadTomes,Tue Feb 12 11:38:36 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 00:00:00 -0800 2013,1,0,"['review', 'ing', 'soon', 'readtome']"
3,5805,c309dff1695ed8558b29ea8dcd7479b8,0da0bcc469c2acd15350f9a8f0a74e2b,5,"What better way to celebrate Guy Fawkes Day, t...",Tue May 20 09:52:36 -0700 2014,Thu Mar 16 09:49:11 -0700 2017,Wed Nov 05 14:22:10 -0800 2014,Wed Nov 05 00:00:00 -0800 2014,2,0,"['well', 'way', 'celebrate', 'guy', 'day', 're..."
4,5805,0f6b8c04f811e05c8978bd6b66ce7685,7d670b6c8cac0c086e21ae2f1af6eccb,4,To note - I am writing this review a quarter c...,Tue Jul 03 16:43:26 -0700 2012,Fri Feb 27 14:46:00 -0800 2015,Tue Jan 01 00:00:00 -0800 1991,NaN,10,0,"['note', 'write', 'review', 'quarter', 'centur..."


In [9]:
def to_list(x):
    x = x.strip('][').split(', ')
#     print(x)
    x = [i.strip("'") for i in x]
    return x

In [10]:
engaged_df['clean_reviews'] = engaged_df['clean_reviews'].apply(lambda x: to_list(x))

In [11]:
book_df = pd.read_csv('./data/books_with_topics.csv')

In [12]:
book_df['processed_description'] = book_df['processed_description'].apply(lambda x: to_list(x))

In [13]:
lda_review_model = gensim.models.ldamodel.LdaModel.load("./data/best_review_model")

In [14]:
lda_review_model.print_topics()

[(0,
  '0.016*"con" + 0.011*"" + 0.009*"para" + 0.005*"solo" + 0.004*"star" + 0.004*"son" + 0.004*"mal" + 0.003*"tan" + 0.003*"phoebe" + 0.003*"ser"'),
 (1,
  '0.021*"book" + 0.010*"story" + 0.009*"read" + 0.007*"life" + 0.007*"graphic" + 0.007*"novel" + 0.007*"one" + 0.007*"make" + 0.006*"like" + 0.005*"time"'),
 (2,
  '0.020*"read" + 0.017*"like" + 0.017*"story" + 0.014*"really" + 0.014*"love" + 0.013*"character" + 0.012*"volume" + 0.012*"get" + 0.012*"one" + 0.012*"book"'),
 (3,
  '0.012*"story" + 0.011*"batman" + 0.007*"one" + 0.006*"man" + 0.006*"character" + 0.005*"make" + 0.005*"new" + 0.005*"doe" + 0.005*"book" + 0.005*"take"'),
 (4,
  '0.010*"vader" + 0.009*"kai" + 0.009*"die" + 0.008*"che" + 0.007*"yang" + 0.006*"fairyland" + 0.006*"tintin" + 0.006*"gertrude" + 0.005*"den" + 0.005*"dan"')]

## Get a recommendation for a User

In [15]:
# Let us choose a random user

In [16]:
c = np.random.choice(engaged_df['user_id'].unique())

In [17]:
c

'fc2875d991f75cf300dd3aedb2784c08'

In [25]:
def generate_recommendations(c, inter_df, engaged_df, book_df, is_debug = True):
    
    #get all the books the user is reviewing
    r_books = engaged_df[engaged_df['user_id'] == c]['book_id'].unique()
    #get all the books the user has interacted with
    r_books = np.concatenate((r_books , inter_df[inter_df['user_id'] == c]['book_id'].unique()))
    
    if is_debug:
        print("These are the books that the user has interacted with:")
        print(book_df[book_df['book_id'].isin(r_books)]['title'].values)
    #Get the distribution of topics
    book_dist = book_df[book_df['book_id'].isin(r_books)]['dom_topic_reviews'].value_counts() 
    book_dist = book_dist/sum(book_dist)
    
    if is_debug:
        
        print(book_dist)
    
    reco = []
    
    count = 0
    
    #Generate a recommnedation using the LDA model
    #If the user has already interacted with the books, just ignore and try to generate a new recommendation
    
    while count< 10:
        idx = random.choices(book_dist.index, weights=book_dist.values)[0]
        id_reco = np.random.choice(book_df[book_df['dom_topic_reviews'] == idx].index, )
        if is_debug:
            print(id_reco)
        if book_df.iloc[id_reco]['book_id'] in engaged_df[engaged_df['user_id'] == c]['book_id']:
            pass
        else:
            reco.append((book_df.iloc[id_reco]['title'], idx))
            count = count + 1
    return reco

In [26]:
recos = generate_recommendations(c, inter_df, engaged_df, book_df)

These are the books that the user has interacted with:
['Saga, Vol. 1 (Saga, #1)'
 'The Authoritative Calvin and Hobbes: A Calvin and Hobbes Treasury'
 'The Indispensable Calvin and Hobbes'
 'Homicidal Psycho Jungle Cat: A Calvin and Hobbes Collection'
 'Darth Vader and Son' 'Fruits Basket, Vol. 10']
2    0.5
1    0.5
Name: dom_topic_reviews, dtype: float64
507
48
128
998
287
621
150
249
561
51


In [ ]:
book_df[book_df['']]

In [63]:
book_df[book_df['book_id']< 1000]

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,work_id,title,title_without_series,processed_description,topic_0_review,topic_1_review,topic_2_review,topic_3_review,topic_4_review,dom_topic_reviews
18,1591169208,1055,['180247'],US,eng,"[{'count': '3926', 'name': 'to-read'}, {'count...",0,False,4.49,B00JDRKY48,...,1959820,"Fullmetal Alchemist, Vol. 1 (Fullmetal Alchemi...","Fullmetal Alchemist, Vol. 1 (Fullmetal Alchemi...","[alchemy, mystical, power, alter, natural, wor...",0.012986,0.055721,0.681300,0.223218,0.026775,2
295,1591169232,293,['180248'],US,eng,"[{'count': '522', 'name': 'mangá'}, {'count': ...",0,False,4.53,B00JDRKY5C,...,2276648,"Fullmetal Alchemist, Vol. 2 (Fullmetal Alchemi...","Fullmetal Alchemist, Vol. 2 (Fullmetal Alchemi...","[alchemical, ritual, wrong, lose, arm, leg, br...",0.000000,0.000000,0.833025,0.140636,0.023239,2
303,1591169259,204,['180249'],US,eng,"[{'count': '1590', 'name': 'to-read'}, {'count...",0,False,4.56,B00JDRKY4I,...,1347600,"Fullmetal Alchemist, Vol. 3 (Fullmetal Alchemi...","Fullmetal Alchemist, Vol. 3 (Fullmetal Alchemi...","[bodyguard, hero, seek, close, childhood, frie...",0.000000,0.047849,0.668484,0.281280,0.000000,2
442,1591169291,204,['180250'],US,eng,"[{'count': '1303', 'name': 'manga'}, {'count':...",0,False,4.55,B00JDRKYCA,...,1347602,"Fullmetal Alchemist, Vol. 4 (Fullmetal Alchemi...","Fullmetal Alchemist, Vol. 4 (Fullmetal Alchemi...","[trap, injure, secret, alchemical, laboratory,...",0.000000,0.089410,0.647369,0.258542,0.000000,2
499,1421504596,116,['180254'],US,eng,"[{'count': '389', 'name': 'mangá'}, {'count': ...",0,False,4.57,B00JDRL1BI,...,1321122,"Fullmetal Alchemist, Vol. 8 (Fullmetal Alchemi...","Fullmetal Alchemist, Vol. 8 (Fullmetal Alchemi...","[alchemical, ritual, wrong, lose, arm, leg, br...",0.000000,0.075848,0.434702,0.464163,0.021315,3
520,142150460X,109,['180255'],US,eng,"[{'count': '1195', 'name': 'to-read'}, {'count...",0,False,4.57,B00JDRL1M2,...,1347603,"Fullmetal Alchemist, Vol. 9 (Fullmetal Alchemi...","Fullmetal Alchemist, Vol. 9 (Fullmetal Alchemi...","[return, central, command, bad, news, greet, f...",0.000000,0.000000,0.415401,0.581082,0.000000,3


In [61]:
list(inter_df[inter_df['user_id'] == "8842281e1d1347389f2ab93d60773d4d"]['book_id'])

[15704307, 59715, 24815, 24816, 13163846, 836610]

In [62]:
book_df[book_df['book_id'].isin(list(inter_df[inter_df['user_id'] == "8842281e1d1347389f2ab93d60773d4d"]['book_id']))]

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,work_id,title,title_without_series,processed_description,topic_0_review,topic_1_review,topic_2_review,topic_3_review,topic_4_review,dom_topic_reviews
4,1607066017,8700,['736247'],US,eng,"[{'count': '78762', 'name': 'to-read'}, {'coun...",0,False,4.24,B015XEABR4,...,19113524,"Saga, Vol. 1 (Saga, #1)","Saga, Vol. 1 (Saga, #1)","[two, soldier, opposite, side, never, end, gal...",0.0,0.064265,0.856246,0.066379,0.000000,2
211,751507954,372,"['913809', '242076']",US,eng,"[{'count': '2480', 'name': 'to-read'}, {'count...",0,False,4.73,B00FWOKE2I,...,1010526,The Authoritative Calvin and Hobbes: A Calvin ...,The Authoritative Calvin and Hobbes: A Calvin ...,"[collection, cartoon, author, win, award, outs...",0.0,0.773800,0.210494,0.014955,0.000000,1
229,751500283,262,"['913810', '242077']",US,eng,"[{'count': '689', 'name': 'comics'}, {'count':...",0,False,4.73,B00FWOKE1Y,...,25602,The Indispensable Calvin and Hobbes,The Indispensable Calvin and Hobbes,"[back, six, year, old, dirty, tricksmeister, m...",0.0,0.795184,0.166870,0.035919,0.000000,1
242,836217691,262,"['242072', '242080']",US,eng,"[{'count': '1581', 'name': 'to-read'}, {'count...",0,False,4.71,B00LUKGF2G,...,2369970,Homicidal Psycho Jungle Cat: A Calvin and Hobb...,Homicidal Psycho Jungle Cat: A Calvin and Hobb...,"[homicidal, jungle, multifarious, adventure, w...",0.0,0.578725,0.352961,0.060708,0.000000,1
245,145210655X,1224,['544810'],US,eng,"[{'count': '5705', 'name': 'to-read'}, {'count...",0,False,4.14,B00V9G41KY,...,18342898,Darth Vader and Son,Darth Vader and Son,"[vader, take, active, role, raise, son, luke, ...",0.0,0.343663,0.556341,0.025339,0.065561,2
268,1595324054,156,['205828'],US,eng,"[{'count': '2411', 'name': 'to-read'}, {'count...",0,False,4.45,0,...,237194,"Fruits Basket, Vol. 10","Fruits Basket, Vol. 10","[world, number, one, shojo, manga, continue, t...",0.0,0.195575,0.703165,0.000000,0.099876,2


In [54]:
print(recos)

[('Hellboy: Weird Tales, Vol. 1 (Hellboy: Weird Tales, #1)', 2), ('Deadly Class, Vol. 1: Reagan Youth', 2), ('Naruto, Vol. 32:  The Search for Sasuke (Naruto, #32)', 2), ('Angel Sanctuary, Vol. 1', 2), ('Skip Beat!, Vol. 04', 2), ('Vampire Knight, Vol. 3 (Vampire Knight, #3)', 2), ('Sailor Moon, #2 (Sailor Moon, #2)', 2), ('Page by Paige', 1), ('Uzumaki, Vol. 1', 2), ('Mouse Guard: Fall 1152 (Mouse Guard, #1)', 2)]


## Get a recommendation for a particular user

In [64]:
c = "4980305f36ab8c2ab831e401a185f28a"

In [71]:
recos = generate_recommendations(c, inter_df, engaged_df, book_df)

These are the books that the user has interacted with:
['Death Note, Vol. 1: Boredom (Death Note, #1)'
 'Naruto, Vol. 01: The Tests of the Ninja (Naruto, #1)'
 'Vampire Knight, Vol. 1 (Vampire Knight, #1)'
 'Black Butler, Vol. 1 (Black Butler, #1)'
 'Blue Exorcist, Vol. 1  (Blue Exorcist, #1)'
 'Death Note, Vol. 2: Confluence (Death Note, #2)'
 'Death Note, Vol. 3: Hard Run (Death Note, #3)'
 'Death Note, Vol. 4: Love (Death Note, #4)'
 'Death Note, Vol. 5: Whiteout (Death Note, #5)'
 'Black Butler, Vol. 2 (Black Butler, #2)'
 'Naruto, Vol. 11: Impassioned Efforts (Naruto, #11)'
 'Death Note, Vol. 6: Give-and-Take (Death Note, #6)'
 'Death Note, Vol. 8: Target (Death Note, #8)' 'Black Cat, Volume 01'
 'Vampire Knight, Vol. 4 (Vampire Knight, #4)'
 'Vampire Knight, Vol. 2 (Vampire Knight, #2)'
 'Vampire Knight, Vol. 3 (Vampire Knight, #3)'
 'Naruto, Vol. 05: Exam Hell (Naruto, #5)'
 'Naruto, Vol. 02: The Worst Client (Naruto, #2)'
 'Vampire Knight, Vol. 5 (Vampire Knight, #5)'
 'Death N

In [73]:
for rec in recos:
    print(f"--> {rec}")

--> ('Ghostopolis', 2)
--> ('Naruto, Vol. 31: Final Battle (Naruto, #31)', 2)
--> ('The Batman Adventures: Mad Love', 2)
--> ('Death Note, Vol. 8: Target (Death Note, #8)', 2)
--> ('Fullmetal Alchemist, Vol. 25 (Fullmetal Alchemist, #25)', 2)
--> ('The Walking Dead, Book Three (The Walking Dead #25-36)', 2)
--> ('Ouran High School Host Club, Vol. 13 (Ouran High School Host Club, #13)', 2)
--> ('Naruto, Vol. 05: Exam Hell (Naruto, #5)', 2)
--> ('Wires and Nerve, Volume 1 (Wires and Nerves, #1)', 2)
--> ('Red River, Vol. 1 (Red River, #1)', 2)


In [75]:
test = ['Death Note, Vol. 1: Boredom (Death Note, #1)',
       'Naruto, Vol. 01: The Tests of the Ninja (Naruto, #1)',
       'Death Note, Vol. 2: Confluence (Death Note, #2)',
       'Death Note, Vol. 3: Hard Run (Death Note, #3)',
       'Death Note, Vol. 5: Whiteout (Death Note, #5)',
       'Death Note, Vol. 6: Give-and-Take (Death Note, #6)',
       'Black Cat, Volume 01', 'The Absolute Sandman, Volume Four',
       'Naruto, Vol. 27: Departure (Naruto, #27)',
       'Batman: Under the Red Hood']

In [76]:
for rec in test:
    print(f"--> {rec}")

--> Death Note, Vol. 1: Boredom (Death Note, #1)
--> Naruto, Vol. 01: The Tests of the Ninja (Naruto, #1)
--> Death Note, Vol. 2: Confluence (Death Note, #2)
--> Death Note, Vol. 3: Hard Run (Death Note, #3)
--> Death Note, Vol. 5: Whiteout (Death Note, #5)
--> Death Note, Vol. 6: Give-and-Take (Death Note, #6)
--> Black Cat, Volume 01
--> The Absolute Sandman, Volume Four
--> Naruto, Vol. 27: Departure (Naruto, #27)
--> Batman: Under the Red Hood
